## Objetivo: 

- Obtener conversión única de los argumentos desde palabras a números 
- Crear matriz (embedding layer) con los vectores de fasttext en las posiciones respectivas a la conversión anterior.

In [1]:
import pandas as pd
import numpy as np
import logging
from itertools import product
import unicodedata
import re

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.utils import np_utils
from keras import regularizers
from keras.models import Sequential
from keras.optimizers import SGD
from keras import regularizers
from keras.models import load_model, Model
from keras.layers import Dense, Dropout, Input, Embedding, Lambda
from keras import backend as K
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

Using TensorFlow backend.


In [3]:
from gensim.models.wrappers import FastText
model = FastText.load_fasttext_format('../../word_vectors/wiki.es')

In [4]:
EMBEDDING_DIM = 300

### Load data

In [15]:
df = {}
for set_ in ['train', 'dev', 'test']:
    filename = '../../data/tarea3/x_'+set_+'_modo.txt'
    with open(filename) as f:
        data = f.readlines()
    df[set_] = [row[:-1] for row in data]

In [20]:
print df['train'][0]
print df['dev'][0]

Que se respete el espíritu de la ley y esta no se pueda interpretar de forma fraudulenta o malintencionada. Implica leyes bien redactadas.
Una sociedad no puede ser democrática si las decisiones no están basadas en posturas informadas, es por esto que el acceso a la información debe ser considerada un pilar de la vida en democracia


In [17]:
df_x = []
for set_ in ['train', 'dev', 'test']:
    df_x += df[set_]

In [19]:
print df_x[0]
print df_x[len(df['train'])]

Que se respete el espíritu de la ley y esta no se pueda interpretar de forma fraudulenta o malintencionada. Implica leyes bien redactadas.
Una sociedad no puede ser democrática si las decisiones no están basadas en posturas informadas, es por esto que el acceso a la información debe ser considerada un pilar de la vida en democracia


In [21]:
import string

# arguments (X) from unicode to str
# ignore puntuaction, ñ, accents
# OJO: solo ignoramos puntuación, mantenemos acentos y ñs
arguments = []
for argument in df_x:
    # esto es para sacarle la puntuación y que sea de forma standard con keras
    in_string =  " ".join(text_to_word_sequence(argument))
    arguments.append(in_string)
print((arguments[indices['tarea10train']]))
print((arguments[indices['tarea10dev']]))
print((arguments[indices['tarea11train']]))

que se respete el espíritu de la ley y esta no se pueda interpretar de forma fraudulenta o malintencionada implica leyes bien redactadas
fortalecer la independencia y neutralidad de las instituciones
respeto e inclusión a un otro en especial a las mujeres la falta de equidad de género promueve la segregación favorecer la co parentalidad cuidados personales para un impacto positivo para la infancia


In [22]:
# words to numbers (start with 1)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(arguments) # ignore blank spaces, commas, points etc.
X = tokenizer.texts_to_sequences(arguments)
print (X[0])

[4, 11, 449, 5, 1860, 1, 2, 61, 3, 82, 15, 11, 221, 4217, 1, 68, 29030, 36, 41044, 377, 57, 64, 9515]


In [23]:
import pickle
pickle.dump(tokenizer.word_index, open("dan_preprocessing_data/dict_word_to_num.p", "wb"))

In [24]:
# embedding matrix
# matrix with fasttext vectors, where vector in index i is the corresponding of word i in the tokenizer
index2word_set = set(model.wv.index2word)
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in index2word_set: 
        # words not found in embedding index will be all-zeros.
        embedding_vector = model[word]
        embedding_matrix[i] = embedding_vector

In [25]:
# save weights matrix
np.savetxt('dan_preprocessing_data/embedding_matrix.txt', embedding_matrix, fmt='%f')
# b = np.loadtxt('my_data/embedding_matrix.txt', dtype=int)

In [26]:
print (len(word_index))
print (embedding_matrix.shape)

42398
(42399, 300)


In [ ]:
##### OJO:
##### sin acentos el vocabulario es de tamaño 38224
##### con acentos hay 3400 palabras más! 41656